This .ipynb is an attempt at automating the checking of our steady states. 
The conclusions here are based off the work I have on my table on trying
to solve for equilibria and characteristic equations for matrices. 

In [2]:
import math
import numpy as np

In [3]:
"""
A couple nice functions that report steady states and their equilibrium. 
"""
def report_steady_state(S: float, L: float, G: float):
    print(f"Sparrow population: {S:.3f}")
    print(f"Locust population: {L:.3f}")
    print(f"Grain: {G:.3f}")

def report_equilibrium(*eigenvalues: float):
    """
    Prints eigenvalues and classifies stability.

    TODO: Due to floating point, an == 0 check is unlikely. We may consider
    having some small value epsilon, and use this to check if things are zero.
    """
    for i, lam in enumerate(eigenvalues, start=1):
        print(f"λ_{i} = {lam}")

    # Use real parts when deciding stability
    eps = 1e-12
    reals = [lam.real for lam in eigenvalues]

    has_pos = any(r > eps for r in reals)
    has_neg = any(r < -eps for r in reals)
    has_zero = any(abs(r) <= eps for r in reals)

    if has_zero:
        print("Classification: equilibrium with a zero eigenvalue (nonhyperbolic)")
    elif has_pos and has_neg:
        print("Classification: unstable saddle equilibrium")
    elif has_pos and not has_neg:
        print("Classification: unstable equilibrium")
    elif has_neg and not has_pos:
        print("Classification: stable equilibrium")
    else:
        print("Classification: unable to classify (unexpected case)")

In [15]:
"""
Input parameters
"""
a1 = 0.4
a2 = 0.6
a3 = 1 / 3 

b1 = 3.0
b2 = 0.15 * b1 
b3 = 0.01
b4 = 1.0
b5 = 2.5

c1 = 0.6
c2 = 3
c3 = 0.015

Eh = 0

def f(L: float) -> float: 
    return b1 + (b2 * (L**2))/(b3**2 + L**2)

def dfdx(L: float) -> float:
    return (2 * b2 * (b3**2) * L)/((b3**2 + L**2) ** 2)

Case A: Extinction Steady State
We set S and L equal to 0. 

In [5]:
"""
It is straightforwards to solve for the quantities at this equilibrium. 
"""

S_ext = 0
L_ext = 0
G_ext = 1/c3

report_steady_state(S_ext, L_ext, G_ext)

Sparrow population: 0.000
Locust population: 0.000
Grain: 66.667


In [6]:
"""
The Jacobian is a diagonal matrix. 
"""
l1_extinction = (a2/a3) - a3 - Eh 
l2_extinction = b1 - b4 
l3_extinction = -c3 

report_equilibrium(l1_extinction, l2_extinction, l3_extinction)

λ_1 = 0.4666666666666668
λ_2 = 2.0
λ_3 = -0.015
Classification: unstable saddle equilibrium


Case B: S = 0, L > 0

In [ ]:
"""
Steady state populations when sparrows are extinct. 
"""
S_no_sparrows = 0 
L_no_sparrows = b3 * (math.sqrt((-1 * b1 + b4)/(b1 + b2 - b4)))
G_no_sparrows = 1 / ((c2 * L_no_sparrows) + c3)

report_steady_state(S_no_sparrows, L_no_sparrows, G_no_sparrows)

ValueError: math domain error

In [ ]:
"""
Eigenvalues when sparrows are extinct. 
"""
l1_no_sparrows = (a1 * L_no_sparrows + a2 * G_no_sparrows - a3 - Eh)
l2_no_sparrows = (dfdx(L_no_sparrows)) * L_no_sparrows
l3_no_sparrows = -1 / G_no_sparrows

report_equilibrium(l1_no_sparrows, l2_no_sparrows, l3_no_sparrows)

NameError: name 'L_no_sparrows' is not defined

Case C: L = 0, S,G > 0

In [7]:
"""
Steady state populations when locusts are extinct. 
"""
S_C = (1 / c1) * (a2 / (a3 + Eh) - c3)
L_C = 0 
G_C = (a3 + Eh) / a2 

report_steady_state(S_C, L_C, G_C)


Sparrow population: 0.725
Locust population: 0.000
Grain: 2.222


In [8]:
"""
Eigenvalues when locusts are extinct.
"""

lambda1_C = b1 - b4 - b5 * S_C

T = a2 / (a3 + Eh)
D = a2 - c3 * (a3 + Eh)

discriminant = T**2 - 4 * D
sqrt_disc = discriminant**0.5 

lambda2_C = (-T + sqrt_disc) / 2
lambda3_C = (-T - sqrt_disc) / 2

report_equilibrium(lambda1_C, lambda2_C, lambda3_C)

λ_1 = 0.1875
λ_2 = (-0.22499999999999995+0.7275816105427624j)
λ_3 = (-0.22500000000000006-0.7275816105427624j)
Classification: unstable saddle equilibrium


Case D: Coexistence. 

In [11]:
def coexistence_equation_scalar(L: float) -> float: 
    """
    It is possible to solve for the values of S and G in terms of L. 

    We build an equation whose root corresponds to the L at a steady state, 
    so we can calculate S and G from that root. 

    Deriving this function: 

    dL/dt = 0 gets us to f(L) - b4 - b5 * S = 0, so S(L) = (f(L) - b4) / b5 

    dG/dt = 0 gets us to G(L) = 1 / (c1 * S(L) + c2 * L + c3)

    dS/dt = 0 gets us to a1 * S * L + a2 * S * G - a3 * S - Eh * S = 0. 
    Since S > 0, we can write this as a1 * L + a2 * G = a3 + Eh. 

    > a1 * L + a2 * G = a3 + Eh <  is true at equilibrium. 
    So then, we can set F(L) = a1*L + a2 / (c1*S(L) + c2*L + c3) - (a3 + Eh)
    and having this value equal to 0 gives us the L at equilibrium. 

    We then use a root solver on F. 
    """
    if L <= 0:
        return float("nan")

    S = (f(L) - b4) / b5
    if S < 0:
        return float("nan")

    denom = c1*S + c2*L + c3
    G = 1.0 / denom
    if G < 0: 
        return float("nan")

    return a1*L + a2*G - (a3 + Eh)

In [16]:
from scipy.optimize import fsolve

def find_coexistence_equilibrium_fsolve():
    """
    Shove our problems at scipy and win. 
    """
    L0 = 1.0  


    L_star = fsolve(coexistence_equation_scalar, L0)[0]

    if L_star <= 0 or not np.isfinite(L_star):
        print("fsolve returned invalid L*")
        return None

    S_star = (f(L_star) - b4) / b5
    denom = c1*S_star + c2*L_star + c3
    G_star = 1.0 / denom

    if S_star < 0 or G_star < 0: 
        print("Invalid result for case D, coexistence. ")
        return None

    return S_star, L_star, G_star

In [17]:
"""
Stability at the coexistence steady state. 
"""
def jacobian_coexistence(S: float, L: float, G: float) -> np.ndarray:
    return np.array([
        [0.0,        a1 * S,          a2 * S],
        [-b5 * L,    dfdx(L) * L,     0.0   ],
        [-c1 * G,   -c2 * G,        -1.0 / G]
    ], dtype=float)

In [18]:
coexistence_result = find_coexistence_equilibrium_fsolve()
if coexistence_result:
    S_D, L_D, G_D = coexistence_result
    report_steady_state(S_D, L_D, G_D)

    J_star = jacobian_coexistence(S_D, L_D, G_D)
    eigvals = np.linalg.eigvals(J_star)

    report_equilibrium(*eigvals)
else:
    print("Could not find a steady state.")


Sparrow population: 0.980
Locust population: 0.506
Grain: 0.471
λ_1 = -0.0007104856307925678
λ_2 = -0.37900339672799654
λ_3 = -1.7417871585515536
Classification: stable equilibrium


/var/folders/bz/90tdbd6n2t7dr40z8nt1g5p80000gn/T/ipykernel_16110/963125692.py:10: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  L_star = fsolve(coexistence_equation_scalar, L0)[0]
